In [2]:
import ifcopenshell
import ifcopenshell.api
from ifcopenshell.api import run
import ifcopenshell.alignment
import numpy as np

model = ifcopenshell.file(schema="IFC4X3_ADD2")

project = run("root.create_entity", model, ifc_class="IfcProject", name="Dummy")

geometric_representation_context = run("context.add_context",model,context_type="Model")
axis_model_representation_subcontext = model.createIfcGeometricRepresentationSubContext(
    ContextIdentifier="Axis",
    ContextType="Model",
    ParentContext=geometric_representation_context,
    TargetView="MODEL_VIEW")


coordinates = [(0.0,0.0),(100.0,0.0),(200.0,200.0)]
radii = [(50.0)]
helper = ifcopenshell.alignment.IfcAlignmentHelper(model)
alignment = helper.add_alignment("Dummy",coordinates,radii)
curve = alignment.Representation.Representations[0].Items[0]
print(curve)
#curve = model.by_type("IfcCompositeCurve")[0] # because helper returns Nothing
s = ifcopenshell.geom.settings()
shape = ifcopenshell.geom.create_shape(s,curve)
verts = shape.verts
verts = np.array(verts).reshape((-1,3))
#x,y,z = verts.T
print(verts)

This is a test
#49=IfcCompositeCurve((#18,#28,#38,#48),.F.)
[[  0.           0.           0.        ]
 [  0.49946588   0.           0.        ]
 [  0.99893176   0.           0.        ]
 ...
 [199.55326413 199.10652827   0.        ]
 [199.77663207 199.55326413   0.        ]
 [200.         200.           0.        ]]
